### [DART] 단일회사 전체 재무제표
* https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019020

In [58]:
import requests
import pandas as pd

# 상장기업 재무정보 - 단일회사 전체 재무제표
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
fs_div = 'OFS'      # CFS:연결재무제표, OFS:재무제표
fyear = '2022'        # 사업연도
rpt_code = '11011'  # 사업보고서
corp_codes = [
    '00126380', #삼성전자
    '00119195', #동화약품
    # '01512654', #영풍문고
    '00141307', #영풍
    # '00141334', #영풍산업
    '00141404', #영풍제지
]

df_fi = pd.DataFrame()
for corp_code in corp_codes:
    params = {
        'corp_code': corp_code,
        'bsns_year': fyear,         
        'reprt_code': rpt_code,        
        'fs_div': fs_div,           
        'crtfc_key': api_key,
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:     # URL GET '200 정상'
        json_data = response.json()
        if json_data['status'] == '013':    # 조회된 데이타가 없습니다.
            print(corp_code, json_data['message'])
        else:
            print(corp_code, json_data['message'])
            df = pd.json_normalize(response.json()['list'])
            df_fi = pd.concat([df_fi, df], axis = 0)    # 데이터 결합        
    else:
        print("URL GET Error")

00126380 정상
00119195 정상
00141307 정상
00141404 정상


#### 재무상태표

In [59]:
sj_div = "BS"  # 재무상태표
df_bs = df_fi.query('sj_div == @sj_div').sort_values('ord')
df_rp = df_bs[df_bs['account_nm'].str.contains("급여")][['rcept_no', 'bsns_year', 'corp_code', 'account_id', 'account_nm', 'thstrm_amount', 'currency']]          # 당기금액
df_rp

,rcept_no,bsns_year,corp_code,account_id,account_nm,thstrm_amount,currency
14,20230307000542,2022,00126380,dart_DepositsForSeveranceInsurance,순확정급여자산,4410223000000,KRW
18,20230712000496,2022,00141404,dart_InvestedAssetForPostemploymentBenefit,순확정급여자산,130152625,KRW
25,20230315001100,2022,00119195,dart_PresentValueOfDefinedBenefitObligation,확정급여채무,73138755,KRW
26,20230713000143,2022,00141307,dart_PostemploymentBenefitObligations,퇴직급여부채,,KRW
30,20230712000496,2022,00141404,dart_PostemploymentBenefitObligations,순확정급여부채,0,KRW


In [71]:
# Null(NA), '', '0' 삭제
idx_list = df_rp[(df_rp['thstrm_amount'].isna()) | ((df_rp['thstrm_amount'] == '')) | (df_rp['thstrm_amount'] == '0')].index
df_rp.drop(index=idx_list, axis=0, inplace=True)
df_rp

,rcept_no,bsns_year,corp_code,account_id,account_nm,thstrm_amount,currency
14,20230307000542,2022,00126380,dart_DepositsForSeveranceInsurance,순확정급여자산,4410223000000,KRW
18,20230712000496,2022,00141404,dart_InvestedAssetForPostemploymentBenefit,순확정급여자산,130152625,KRW
25,20230315001100,2022,00119195,dart_PresentValueOfDefinedBenefitObligation,확정급여채무,73138755,KRW
